--- Day 7: Handy Haversacks ---

You land at the regional airport in time for your next flight. In fact, it looks like you'll even have time to grab some food: all flights are currently delayed due to issues in luggage processing.

Due to recent aviation regulations, many rules (your puzzle input) are being enforced about bags and their contents; bags must be color-coded and must contain specific quantities of other color-coded bags. Apparently, nobody responsible for these regulations considered how long they would take to enforce!

For example, consider the following rules:

```
light red bags contain 1 bright white bag, 2 muted yellow bags.
dark orange bags contain 3 bright white bags, 4 muted yellow bags.
bright white bags contain 1 shiny gold bag.
muted yellow bags contain 2 shiny gold bags, 9 faded blue bags.
shiny gold bags contain 1 dark olive bag, 2 vibrant plum bags.
dark olive bags contain 3 faded blue bags, 4 dotted black bags.
vibrant plum bags contain 5 faded blue bags, 6 dotted black bags.
faded blue bags contain no other bags.
dotted black bags contain no other bags.
```

These rules specify the required contents for 9 bag types. In this example, every faded blue bag is empty, every vibrant plum bag contains 11 bags (5 faded blue and 6 dotted black), and so on.

You have a shiny gold bag. If you wanted to carry it in at least one other bag, how many different bag colors would be valid for the outermost bag? (In other words: how many colors can, eventually, contain at least one shiny gold bag?)

In the above rules, the following options would be available to you:

* A bright white bag, which can hold your shiny gold bag directly.
* A muted yellow bag, which can hold your shiny gold bag directly, plus some other bags.
* A dark orange bag, which can hold bright white and muted yellow bags, either of which could then hold your shiny gold bag.
* A light red bag, which can hold bright white and muted yellow bags, either of which could then hold your shiny gold bag.

So, in this example, the number of bag colors that can eventually contain at least one shiny gold bag is 4.

How many bag colors can eventually contain at least one shiny gold bag? (The list of rules is quite long; make sure you get all of it.)


In [1]:
#!fsharp
open System.Text.RegularExpressions

type Rule = { container: string; content: Map<string, int> }

let (|Regex|_|) (p:string) (s:string)  =
    let m = Regex.Match(s, p)
    if m.Success then Some(List.tail [ for g in m.Groups -> g.Value ])
    else None

let parseContent (content: string) = 
    if content = "no other bags" then 
        Map.empty;
    else
        content.Split(",")
        |> Seq.fold 
            ( fun acc part -> 
                match part with
                | Regex @"^\s*([0-9])+\s+(.+)\s+bags?\s*$" [count; bag] ->
                    acc.Add( bag, int count )
                | _ -> acc
            )
            Map.empty

let parse (input: string) = 
    input.Replace("\r", "").Split("\n")
    |> Seq.map
        (fun item ->
            match item with
            | Regex @"^(.+)\s+bags\scontain\s(.+)\.$" [container; content] ->
                Some { container = container; content = parseContent content };
            | _ ->
                None
        )
    |> Seq.choose id

let testInput="""light red bags contain 1 bright white bag, 2 muted yellow bags.
dark orange bags contain 3 bright white bags, 4 muted yellow bags.
bright white bags contain 1 shiny gold bag.
muted yellow bags contain 2 shiny gold bags, 9 faded blue bags.
shiny gold bags contain 1 dark olive bag, 2 vibrant plum bags.
dark olive bags contain 3 faded blue bags, 4 dotted black bags.
vibrant plum bags contain 5 faded blue bags, 6 dotted black bags.
faded blue bags contain no other bags.
dotted black bags contain no other bags."""

parse testInput

index,container,content
0,light red,"[ [bright white, 1], [muted yellow, 2] ]"
1,dark orange,"[ [bright white, 3], [muted yellow, 4] ]"
2,bright white,"[ [shiny gold, 1] ]"
3,muted yellow,"[ [faded blue, 9], [shiny gold, 2] ]"
4,shiny gold,"[ [dark olive, 1], [vibrant plum, 2] ]"
5,dark olive,"[ [dotted black, 4], [faded blue, 3] ]"
6,vibrant plum,"[ [dotted black, 6], [faded blue, 5] ]"
7,faded blue,[ ]
8,dotted black,[ ]


In [1]:
#!fsharp
let findBags (bag: string) (rules: seq<Rule>) =
    let rec inner (found: Set<string>) target =
        rules
        |> Seq.fold
            (fun (found:Set<string>) v ->
                if (v.content.ContainsKey target) && not (found.Contains v.container) then
                    inner (found.Add v.container) v.container
                else
                    found
            )
            found
    inner Set.empty bag

findBags "shiny gold" (parse testInput)

index,value
0,bright white
1,dark orange
2,light red
3,muted yellow


In [1]:
#!fsharp
let taskInput = System.IO.File.ReadAllText "task.txt"

In [1]:
#!fsharp
findBags "shiny gold" (parse taskInput) |> (fun f -> f.Count)

337

In [1]:
#!fsharp
let rec countSize target (rules: seq<Rule>) =
    let rule = Seq.find ( fun f -> f.container = target ) rules
    rule.content 
    |> Map.fold 
        ( fun acc bag count ->
            acc + count + count * ( countSize bag rules )            
        )
        0

parse testInput |> countSize "shiny gold"



32

In [1]:
#!fsharp
parse taskInput |> countSize "shiny gold"

50100